In [1]:
import os
import polars as pl
from polars import col as c
from typing import Union
import networkx as nx

import numpy as np
import pandapower as pp
from general_function import duckdb_to_dict
from polars_function import (
    get_transfo_admittance, get_transfo_impedance, get_transfo_conductance, get_transfo_imaginary_component)
from networkx_function import (
    generate_tree_graph_from_edge_data, get_all_edge_data)


from twindigrid_changes.schema import ChangesSchema
from twindigrid_sql.entries.equipment_class import EXTERNAL_NETWORK, TRANSFORMER, SWITCH, BRANCH, EXTERNAL_NETWORK, TRANSFORMER

from general_function import pl_to_dict


os.chdir(os.getcwd().replace("/src", ""))


In [2]:
file_name = ".cache/input_data/boisy_grid.db"

In [3]:
schema = duckdb_to_dict(file_name)

Read and validate tables from boisy_grid.db file: 100%|██████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 300.94it/s]


In [4]:
schema["branch"]

diff,uuid,section,current_limit,length,is_underground
str,str,f64,f64,f64,bool
"""+""","""751c0012-5ef7-5e8b-9e9c-c30163…",null,305.0,0.296,null
"""+""","""73a079f9-4f0c-5507-9188-962ab3…",null,305.0,0.457,null
"""+""","""1539b17d-745a-5fd8-bbbd-1edbdc…",null,305.0,0.297,null
"""+""","""49d4f814-3b20-553b-a03d-ae6486…",null,305.0,0.163,null
"""+""","""68f19b75-bb7a-5df8-a5e1-cd42c2…",null,280.0,0.279,null
…,…,…,…,…,…
"""+""","""c88cfea3-be41-59fe-9d1e-fb2b80…",null,280.0,0.02,null
"""+""","""e176ca4a-d4d5-5758-b23d-6f720e…",null,460.0,0.02,null
"""+""","""85874642-a636-588e-8ef2-83880f…",null,460.0,0.02,null


In [5]:
node_data

NameError: name 'node_data' is not defined

In [ ]:
grid_data["edge_data"] 

KeyError: 'edge_data'

In [ ]:
transformer

diff,uuid,rated_s,diff_right,uuid_right,timestamp,heartbeat,source_fk,r,r0,x,x0,b,b0,g,g0,side,vn_hv,vn_lv,u_of_edge,v_of_edge,v_base_hv,v_base_lv,i_base,n_transfo,z_base,eq_fk,i_max_pu,r_pu,x_pu,type
str,str,f64,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",str,f64,f64,f64,f64,f64,f64,f64,f64,str,i32,i32,i64,i64,i32,i32,f64,f64,f64,str,f64,f64,f64,str
"""+""","""940b55d4-0fb9-5042-9ee7-6e557f…",400.0,"""+""","""083f501b-db00-4382-b8c6-20e5c3…",2002-08-02 00:00:00 UTC,2025-06-23 11:01:28.318885 UTC,"""scada""",0.00354,0.00354,0.018405,0.018405,0.043014,0.043014,0.00246,0.00246,"""t2""",11900,420,2,123,11500,400,1443.375673,0.985507,0.16,"""940b55d4-0fb9-5042-9ee7-6e557f…",0.4,0.022126,0.115032,"""transformer"""
"""+""","""7408dff2-59bf-5db2-80e5-32f7aa…",630.0,"""+""","""2b3903c7-a92f-4bf9-b261-879ffa…",2008-10-01 00:00:00 UTC,2025-06-23 11:01:28.318885 UTC,"""scada""",0.001898,0.001898,0.010867,0.010867,0.06416,0.06416,0.004025,0.004025,"""t2""",11900,420,6,124,11500,400,1443.375673,0.985507,0.16,"""7408dff2-59bf-5db2-80e5-32f7aa…",0.63,0.011864,0.067922,"""transformer"""
"""+""","""20fe9341-aa89-5f66-956c-0b203d…",630.0,"""+""","""526f5f67-9569-42f2-8208-80e2d9…",2005-08-19 00:00:00 UTC,2025-06-23 11:01:28.318885 UTC,"""scada""",0.001924,0.001924,0.011743,0.011743,0.064194,0.064194,0.003424,0.003424,"""t2""",11900,420,17,126,11500,400,1443.375673,0.985507,0.16,"""20fe9341-aa89-5f66-956c-0b203d…",0.63,0.012025,0.073396,"""transformer"""
"""+""","""4b1761bb-d4c5-5cca-95b5-921b7c…",630.0,"""+""","""75f52a11-6b5f-4576-9a43-a5e28d…",2006-05-05 00:00:00 UTC,2025-06-23 11:01:28.318885 UTC,"""scada""",0.001927,0.001927,0.011686,0.011686,0.064191,0.064191,0.003486,0.003486,"""t2""",11900,420,21,127,11500,400,1443.375673,0.985507,0.16,"""4b1761bb-d4c5-5cca-95b5-921b7c…",0.63,0.012042,0.073039,"""transformer"""
"""+""","""080bf02a-f107-5db0-83d6-44233d…",25000.0,"""+""","""a8e83934-bb9d-495c-a223-e272fb…",1992-05-03 00:00:00 UTC,2025-06-23 11:01:28.318885 UTC,"""scada""",0.019259,0.019259,0.622786,0.622786,0.000063,0.000063,0.000117,0.000117,"""t2""",132000000,11900,125,11,132000,11500,50.204371,1.194643,132.25,"""080bf02a-f107-5db0-83d6-44233d…",25.0,0.000146,0.004709,"""transformer"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""d8a5d1b9-e9b9-5f5f-bc61-7f802d…",400.0,"""+""","""b1eed1b7-5e8d-480b-b63b-0c2a8f…",2008-10-29 00:00:00 UTC,2025-06-23 11:01:28.318885 UTC,"""scada""",0.003538,0.003538,0.01629,0.01629,0.042986,0.042986,0.002908,0.002908,"""t2""",11900,420,106,149,11500,400,1443.375673,0.985507,0.16,"""d8a5d1b9-e9b9-5f5f-bc61-7f802d…",0.4,0.022112,0.101813,"""transformer"""
"""+""","""239542ba-77cd-50a4-9885-cbb0e6…",630.0,"""+""","""a20912ce-6e0a-4fc9-8b7a-bf8a46…",2000-01-08 00:00:00 UTC,2025-06-23 11:01:28.318885 UTC,"""scada""",0.001953,0.001953,0.012419,0.012419,0.064186,0.064186,0.003583,0.003583,"""t2""",11900,420,110,150,11500,400,1443.375673,0.985507,0.16,"""239542ba-77cd-50a4-9885-cbb0e6…",0.63,0.012206,0.077621,"""transformer"""
"""+""","""6952857c-a1f7-55c4-bca4-a79d75…",630.0,"""+""","""a161c382-a4a9-4ad5-bef3-b62348…",2005-11-19 00:00:00 UTC,2025-06-23 11:01:28.318885 UTC,"""scada""",0.001925,0.001925,0.011687,0.011687,0.064192,0.064192,0.003475,0.003475,"""t2""",11900,420,116,151,11500,400,1443.375673,0.985507,0.16,"""6952857c-a1f7-55c4-bca4-a79d75…",0.63,0.012031,0.073041,"""transformer"""


In [ ]:
node_data

node_id,diff,uuid,base_voltage_fk,cn_fk,v_base,i_base,p_node_pu
u32,str,str,i32,str,i32,f64,f64
0,"""+""","""4461087d-937e-5211-a058-13fdfb…",11500,"""4461087d-937e-5211-a058-13fdfb…",11500,50.204371,null
1,"""+""","""0cc6f107-c742-5580-86b6-df40f9…",11500,"""0cc6f107-c742-5580-86b6-df40f9…",11500,50.204371,null
2,"""+""","""c7204c7e-5611-5355-82e6-6bf882…",11500,"""c7204c7e-5611-5355-82e6-6bf882…",11500,50.204371,null
3,"""+""","""b982586f-baf0-5acf-9561-ecf0ca…",11500,"""b982586f-baf0-5acf-9561-ecf0ca…",11500,50.204371,null
4,"""+""","""04aef837-419e-5dd8-b8a4-c0cc40…",11500,"""04aef837-419e-5dd8-b8a4-c0cc40…",11500,50.204371,null
…,…,…,…,…,…,…,…
1013,"""+""","""84836ce4-6956-5848-831d-1b5c94…",400,"""84836ce4-6956-5848-831d-1b5c94…",400,1443.375673,null
1014,"""+""","""3d7be886-c3cc-504a-835e-058f73…",400,"""3d7be886-c3cc-504a-835e-058f73…",400,1443.375673,null
1015,"""+""","""201df1ad-850f-526f-a8da-62863d…",400,"""201df1ad-850f-526f-a8da-62863d…",400,1443.375673,0.004756
